# Hypothesis / Property based testing notes

* Info on [settings](http://hypothesis.readthedocs.io/en/latest/settings.html)
* Scientific [stack](http://hypothesis.readthedocs.io/en/latest/numpy.html)

In [1]:
import numpy as np

In [2]:
from hypothesis import given, settings, Verbosity, assume,note
from hypothesis.strategies import lists,integers, floats
from hypothesis.extra.numpy import arrays

In [3]:
def is_degenerate(v0, v1, v2):
    if np.linalg.norm(v1-v0) == 0:
        return True
    if np.linalg.norm(v2-v1) == 0:
        return True
    if np.linalg.norm(v0-v2) == 0:
        return True
    return False

def orient(a, b, c): 
    return (b[0] - a[0]) * (c[1] - a[1]) - (c[0] - a[0]) * (b[1] - a[1])

def is_in_triangle(v0, v1, v2, pt):
    a = orient(v0, v1, pt) 
    b = orient(v1, v2, pt) 
    c = orient(v2, v0, pt) 
    if (a >= 0) and (b >= 0) and (c >= 0): 
        return True
    return False

def rot2(th):
    st = np.sin(th)
    ct = np.cos(th)
    return np.array([[ct, -st], [st, ct]])

def scale2(v):
    v_x, v_y = v
    return np.array([[v_x, 0], [0, v_y]])

def vnorm(v):
    return np.sqrt(np.dot(v.T, v))

def transform(x, a, b): 
    assert a.shape[0] == a.shape[1]
    assert b.shape[0] == a.shape[0]
    assert b.shape[1] == 1
    return np.dot(a, x) + b 

In [4]:
def vec2(elements=None):
    if elements is None:
        elements=floats(0,1)
    return arrays(dtype=np.float, shape=(2,1), elements=elements)

def angle_rad():
    return floats(0, 2*np.pi)

# Testing Approach

We care specifically about is_in_triangle being invariant to rotations, translations and scalings (and probably shearing). Rather than trying to generate big test vectors, the approach here is to instead state those properties in two parts. 

The first is to use barycentric co-ordinates to find a point that is "clearly-in", and test that point for being in/out over a range of rotations, translations and scale. We can use a similar approach to consider points that are "clearly-out"

The second is to show that on common boundaries, where the triangles have common winding order, that there are no gaps, cracks or seams. This is done by generated quads (two triangles) that are then subjected to transformsions

In [5]:
eps = 100*np.finfo(np.float).eps

In [6]:
with settings(max_examples=5000, max_iterations=10000):
    @given( x=vec2(), y=vec2(elements=floats(-20,20)), s=floats(min_value=1.0, max_value=1000.0), th=angle_rad())
    def test_clearly_in_tri(x,y,s,th):
        assume(x[0] > eps)
        assume(x[1] > eps)
        assume((x[0] + x[1]) < (1-eps))
        u = np.array([[0, 0]]).T
        v = np.array([[1, 0]]).T
        w = np.array([[0, 1]]).T
        res = is_in_triangle(u, v, w, x)
        assert res
        
        A = rot2(th)
        u_bar = s*np.dot(A, u+y)
        v_bar = s*np.dot(A, v+y)
        w_bar = s*np.dot(A, w+y)
        x_bar = s*np.dot(A, x+y)

        res_bar = is_in_triangle(u_bar, v_bar, w_bar, x_bar)
        assert res_bar
        
    
test_clearly_in_tri()

In [7]:
with settings():
    @given( x=vec2(elements=floats(eps, 1-eps)), y=vec2(), th=angle_rad())
    def test_clearly_in_quad(x,y,th):
        u = np.array([[0, 0]]).T
        v = np.array([[1, 0]]).T
        w = np.array([[0, 1]]).T
        p = np.array([[1, 1]]).T
        r1 = is_in_triangle(u, v, w, x)
        r2 = is_in_triangle(w, v, p, x)
        res = r1 or r2
        assert res
        
        note('x=%s, th=%s' % (str(x.T), str(th)))
        
        A = rot2(th)
        u_bar = np.dot(A, u+y)
        v_bar = np.dot(A, v+y)
        w_bar = np.dot(A, w+y)
        p_bar = np.dot(A, p+y)
        x_bar = np.dot(A, x+y)
        r1_bar = is_in_triangle(u_bar, v_bar, w_bar, x_bar)
        r2_bar = is_in_triangle(w_bar, v_bar, p_bar, x_bar)
        res_bar = r1_bar or r2_bar
        assert res_bar
        
    
test_clearly_in_quad()

In [19]:
with settings():#verbosity=Verbosity.verbose):
    @given( x=vec2(elements=floats(-1, 2)), y=vec2(), th=angle_rad())
    def test_clearly_out_quad(x,y,th):
        assume((x[0] < -eps) or (x[0] > 1+eps))
        assume((x[1] < -eps) or (x[1] > 1+eps))
        u = np.array([[0, 0]]).T
        v = np.array([[1, 0]]).T
        w = np.array([[0, 1]]).T
        p = np.array([[1, 1]]).T
        r1 = is_in_triangle(u, v, w, x)
        r2 = is_in_triangle(w, v, p, x)
        res = r1 or r2
        assert not res
        
        A = rot2(th)
        u_bar = np.dot(A, u+y)
        v_bar = np.dot(A, v+y)
        w_bar = np.dot(A, w+y)
        p_bar = np.dot(A, p+y)
        x_bar = np.dot(A, x+y)
        r1_bar = is_in_triangle(u_bar, v_bar, w_bar, x_bar)
        r2_bar = is_in_triangle(w_bar, v_bar, p_bar, x_bar)
        res_bar = r1_bar or r2_bar
        assert not res_bar
    
test_clearly_out_quad()

In [15]:
def unit_quad():
    verts = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])
    tris  = np.array([[0, 1, 2], [1, 3, 2]])
    return verts, tris

verts, _ = unit_quad()

In [16]:
a,b,c,d = verts

In [17]:
a

array([0, 0])